In [2]:
## Import useful classe
from ConnectFour_class import Board,TicTacToe


In [3]:
## TASK 1
class ConnectFour(TicTacToe):
    # Inherit the tictactoe class
    def __init__(self, height=6, width=7, k=4):
        super().__init__(height, width, k)

    #Override the class actions , to introduce the new restriction
    def actions(self,board):
        moves=super().actions(board)
        allowed_moves=[]
        # Restiction: If the position is the lowest of the row , it is not legal
        for move in moves.items():
            if board[(move[0][0],move[0][1]+1)]!='.':
                allowed_moves.append(move)

        return dict(allowed_moves)









In [20]:
## Task 2:
from collections import defaultdict
import numpy as np
import copy
class MTSNode():

    def __init__(self,state,game,parent=None,parent_action=None):
        self.state=state
        self.game=game
        self.parent=parent
        self.parent_action=parent_action
        self.children=[]
        self.n_visits=0
        self.results={}
        self.results[1]=0 ## win
        self.results[-1]=0 ## lose
        self.results[0]=0 ## draw
        self.untried_actions=[]
        self.untried_actions=list(self.possible_actions())
        

    def possible_actions(self):
        self.untried_actions=self.game.actions(self.state)
        return self.untried_actions

    def p(self):
        return self.results[1]-self.results[-1]

    def expand(self):
        move=self.untried_actions.pop()
        ## Copying the state
        next_state=copy.deepcopy(self.state)
        next_state.current_player=self.state.current_player
        # Do the selected move and create a new node
        next_state=self.game.make_move(next_state,move)
        child_node=MTSNode(state=next_state,game=self.game,parent=self,parent_action=move)
        self.children.append(child_node)
        return child_node

    def is_terminal(self):
        return self.game.end(self.state)[0]

    def simulation(self):
        ##Copying the actual state
        current_node=copy.deepcopy(self)
        current_node.state.current_player=self.state.current_player
        ## while it is not a end node, simulate new nodes
        while not current_node.is_terminal():
            move=current_node.simulation_policy()
            current_node.game.make_move(current_node.state,move)
            current_node.untried_actions=list(current_node.possible_actions())
    
        return self.game.utility(current_node.state)

    # Propagations of simulation results
    def backpropagate(self,result):
        self.n_visits+=1
        self.results[result]+=1
        if self.parent:
            self.parent.backpropagate(result)
        
    ## We have create all  child nodes from a parent node
    def is_fully_expanded(self):
        return len(self.untried_actions)==0

    # Chose the node with the best ratio
    def best_child(self,c=2**-0.5):
        param=[]
        for child in self.children:
            try:
                p=child.p()/child.n_visits+c*np.sqrt(2*np.log(self.p())/child.p())
                param.append(p)
            except ZeroDivisionError:
                param.append(0)

        return self.children[np.argmax(param)]

    # We take a random node into the posible nodes to simulate
    def simulation_policy(self):
        possible=self.untried_actions
        return possible[np.random.randint(len(possible))]

    # Create new child of a parent node ande return the best one
    def tree_policy(self):
        while not self.is_terminal():
            if not self.is_fully_expanded():
                 self.expand()
            else:
                break
        
        return self.best_child()
            
    # Use the MCTS algoritm to select the best move to do
    def best_action(self):
        depth=100
        for _ in range(depth):
            # Expand the current node and take the best child
            v=self.tree_policy()
            #Simulate  the rest of moves until a end state
            reward=v.simulation()
            # Back propagate the result to the first node
            v.backpropagate(reward)

        #when doing this depth times , take the best child
        return self.best_child()

    # def __str__(self) :
    #     print('parent move',self.parent_action)
    #     return self.state.__str__()


def MCTS(game,state):

    root=MTSNode(state,game)
    select_move=root.best_action().parent_action

    return select_move

In [25]:
import random
def random_player(game, state):
    return random.choice(list(game.actions(state)))

def player(search_algorithm):
  
    return lambda game, state: search_algorithm(game, state)

def play_game(game, strategies: dict):
    board=game.initial
    flag=False
    while flag==False:
        move=strategies[board.current_player](game,board)
        print(move)
        print('{}=>{}'.format(board.current_player,move))
        board=game.make_move(board,move)
        print(board)
        flag=game.end(board)[0] 
        
        
    utility=game.end(board)[1]  
    return utility

h=play_game(ConnectFour(),dict(X=random_player,O=player(MCTS)))
print(h)


(5, 5)
X=>(5, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . X .



C:\Users\pablo\AppData\Local\Temp\ipykernel_4516\1742439537.py:68: RuntimeWarning: invalid value encountered in log
  p=child.p()/child.n_visits+c*np.sqrt(2*np.log(self.p())/child.p())
C:\Users\pablo\AppData\Local\Temp\ipykernel_4516\1742439537.py:68: RuntimeWarning: divide by zero encountered in log
  p=child.p()/child.n_visits+c*np.sqrt(2*np.log(self.p())/child.p())
C:\Users\pablo\AppData\Local\Temp\ipykernel_4516\1742439537.py:68: RuntimeWarning: invalid value encountered in sqrt
  p=child.p()/child.n_visits+c*np.sqrt(2*np.log(self.p())/child.p())


(3, 5)
O=>(3, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . X .

(2, 5)
X=>(2, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . X O . X .

(1, 5)
O=>(1, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O X O . X .

(6, 5)
X=>(6, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O X O . X X

(1, 4)
O=>(1, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . . . . .
. O X O . X X

(3, 4)
X=>(3, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . X . . .
. O X O . X X

(6, 4)
O=>(6, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . X . . O
. O X O . X X

(5, 4)
X=>(5, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . X . X O
. O X O . X X

(6, 3)
O=>(6, 3)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . O
. O . X . X O
. O X O . X X

(3, 3)
X=>(3, 3)
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . O
. O . X .

In [31]:
def query_player(game,state):
    possible_moves=game.actions(state)
    moves=[move for move in possible_moves.keys()]
    # Save all legal moves in a list
    # Create a dictionari with key: number of the move , value: actual move
    player_move={}
    for i in range(len(moves)):
        player_move[i]=moves[i]
    # Print the explanation text
    print('The legal moves are:\n',player_move)
    print('Chose one of these moves by its number')
    # Take the number given by keyboard
    num=int(input('Number:'))
    print('The chosen move is {}'.format(player_move[num]))
    
    #Return the chosen move as a tuple(x,y)
    return player_move[num]


In [32]:
play_game(ConnectFour(),dict(X=random_player,O=player(query_player)))

(3, 5)
X=>(3, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .

The legal moves are:
 {0: (3, 4), 1: (0, 5), 2: (2, 5), 3: (6, 5), 4: (4, 5), 5: (1, 5), 6: (5, 5)}
Chose one of these moves by its number
The chosen move is (3, 4)
(3, 4)
O=>(3, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . . X . . .

(6, 5)
X=>(6, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . . X . . X

The legal moves are:
 {0: (0, 5), 1: (2, 5), 2: (4, 5), 3: (3, 3), 4: (1, 5), 5: (6, 4), 6: (5, 5)}
Chose one of these moves by its number


ValueError: invalid literal for int() with base 10: ''